# ध्यान तंत्र और ट्रांसफॉर्मर्स

पुनरावर्ती नेटवर्क्स (Recurrent Networks) की एक बड़ी कमी यह है कि अनुक्रम में सभी शब्दों का परिणाम पर समान प्रभाव होता है। यह नामित इकाई पहचान (Named Entity Recognition) और मशीन अनुवाद (Machine Translation) जैसे अनुक्रम-से-अनुक्रम कार्यों के लिए मानक LSTM एन्कोडर-डिकोडर मॉडल्स के साथ उप-इष्टतम प्रदर्शन का कारण बनता है। वास्तविकता में, इनपुट अनुक्रम के कुछ विशिष्ट शब्दों का अनुक्रमिक आउटपुट पर अन्य शब्दों की तुलना में अधिक प्रभाव होता है।

मशीन अनुवाद जैसे अनुक्रम-से-अनुक्रम मॉडल पर विचार करें। इसे दो पुनरावर्ती नेटवर्क्स द्वारा लागू किया जाता है, जहां एक नेटवर्क (**एन्कोडर**) इनपुट अनुक्रम को छिपी हुई स्थिति (hidden state) में संक्षेपित करता है, और दूसरा नेटवर्क (**डिकोडर**) इस छिपी हुई स्थिति को अनुवादित परिणाम में बदलता है। इस दृष्टिकोण की समस्या यह है कि नेटवर्क की अंतिम स्थिति को वाक्य की शुरुआत को याद रखने में कठिनाई होती है, जिससे लंबे वाक्यों पर मॉडल की गुणवत्ता खराब हो जाती है।

**ध्यान तंत्र (Attention Mechanisms)** प्रत्येक इनपुट वेक्टर के संदर्भ प्रभाव को RNN के प्रत्येक आउटपुट भविष्यवाणी पर भारित करने का एक साधन प्रदान करते हैं। इसे लागू करने का तरीका यह है कि इनपुट RNN की मध्यवर्ती अवस्थाओं और आउटपुट RNN के बीच शॉर्टकट्स बनाए जाते हैं। इस प्रकार, जब आउटपुट प्रतीक $y_t$ उत्पन्न किया जाता है, तो हम सभी इनपुट छिपी हुई अवस्थाओं $h_i$ को विभिन्न भार गुणांक $\alpha_{t,i}$ के साथ ध्यान में रखेंगे।

![एन्कोडर/डिकोडर मॉडल जिसमें एक एडिटिव ध्यान परत है](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) में एडिटिव ध्यान तंत्र के साथ एन्कोडर-डिकोडर मॉडल, [इस ब्लॉग पोस्ट](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) से उद्धृत*

ध्यान मैट्रिक्स $\{\alpha_{i,j}\}$ यह दर्शाएगा कि आउटपुट अनुक्रम में किसी दिए गए शब्द को उत्पन्न करने में कौन से इनपुट शब्द कितनी भूमिका निभाते हैं। नीचे इस प्रकार की मैट्रिक्स का एक उदाहरण दिया गया है:

![RNNsearch-50 द्वारा पाए गए एक नमूना संरेखण को दिखाने वाली छवि, Bahdanau - arviz.org से ली गई](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3) से ली गई आकृति]*

ध्यान तंत्र प्राकृतिक भाषा प्रसंस्करण (Natural Language Processing) में वर्तमान या निकट वर्तमान की अत्याधुनिक स्थिति के लिए जिम्मेदार हैं। हालांकि, ध्यान जोड़ने से मॉडल के पैरामीटरों की संख्या में काफी वृद्धि होती है, जिससे RNNs के साथ स्केलिंग समस्याएं उत्पन्न होती हैं। RNNs को स्केल करने की एक प्रमुख बाधा यह है कि मॉडल की पुनरावर्ती प्रकृति प्रशिक्षण को बैच और समानांतर बनाने में चुनौतीपूर्ण बनाती है। RNN में अनुक्रम के प्रत्येक तत्व को क्रमिक क्रम में संसाधित करना पड़ता है, जिसका अर्थ है कि इसे आसानी से समानांतर नहीं किया जा सकता।

ध्यान तंत्रों को अपनाने और इस बाधा ने उन अत्याधुनिक ट्रांसफॉर्मर मॉडलों के निर्माण का मार्ग प्रशस्त किया, जिन्हें हम आज BERT से OpenGPT3 तक उपयोग करते हैं।

## ट्रांसफॉर्मर मॉडल्स

पिछली भविष्यवाणी के संदर्भ को अगले मूल्यांकन चरण में अग्रेषित करने के बजाय, **ट्रांसफॉर्मर मॉडल्स** **पोजिशनल एन्कोडिंग्स** और **ध्यान** का उपयोग करते हैं ताकि दिए गए इनपुट के संदर्भ को एक निर्दिष्ट पाठ विंडो के भीतर कैप्चर किया जा सके। नीचे दी गई छवि दिखाती है कि पोजिशनल एन्कोडिंग्स और ध्यान का उपयोग करके किसी विंडो के भीतर संदर्भ को कैसे कैप्चर किया जा सकता है।

![ट्रांसफॉर्मर मॉडल्स में मूल्यांकन कैसे किया जाता है, यह दिखाने वाला एनिमेटेड GIF।](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

चूंकि प्रत्येक इनपुट स्थिति को स्वतंत्र रूप से प्रत्येक आउटपुट स्थिति पर मैप किया जाता है, ट्रांसफॉर्मर्स RNNs की तुलना में बेहतर समानांतरता प्रदान कर सकते हैं, जिससे बड़े और अधिक अभिव्यक्तिपूर्ण भाषा मॉडल सक्षम होते हैं। प्रत्येक ध्यान हेड का उपयोग शब्दों के बीच विभिन्न संबंधों को सीखने के लिए किया जा सकता है, जो डाउनस्ट्रीम प्राकृतिक भाषा प्रसंस्करण कार्यों में सुधार करता है।

## सरल ट्रांसफॉर्मर मॉडल बनाना

Keras में बिल्ट-इन ट्रांसफॉर्मर लेयर नहीं है, लेकिन हम अपना खुद का बना सकते हैं। पहले की तरह, हम AG News डेटासेट के टेक्स्ट वर्गीकरण पर ध्यान केंद्रित करेंगे, लेकिन यह उल्लेख करना महत्वपूर्ण है कि ट्रांसफॉर्मर मॉडल अधिक कठिन NLP कार्यों में सर्वश्रेष्ठ परिणाम दिखाते हैं।


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

केरस में नई लेयर्स को `Layer` क्लास को सबक्लास करना चाहिए और `call` मेथड को लागू करना चाहिए। चलिए **Positional Embedding** लेयर से शुरू करते हैं। हम [आधिकारिक केरस दस्तावेज़](https://keras.io/examples/nlp/text_classification_with_transformer/) से कुछ कोड का उपयोग करेंगे। हम मान लेंगे कि हम सभी इनपुट अनुक्रमों को `maxlen` लंबाई तक पैड करते हैं।


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

यह लेयर दो `Embedding` लेयर्स से बनी होती है: एक टोकन को एम्बेड करने के लिए (जैसा कि हमने पहले चर्चा की है) और दूसरी टोकन की पोजीशन को एम्बेड करने के लिए। टोकन की पोजीशन को 0 से `maxlen` तक के प्राकृतिक संख्याओं के अनुक्रम के रूप में `tf.range` का उपयोग करके बनाया जाता है, और फिर इसे एम्बेडिंग लेयर में पास किया जाता है। इसके बाद, दो प्राप्त एम्बेडिंग वेक्टर को जोड़ा जाता है, जिससे इनपुट का पोजीशनली-एम्बेडेड प्रतिनिधित्व तैयार होता है, जिसका आकार `maxlen`$\times$`embed_dim` होता है।

अब, आइए ट्रांसफॉर्मर ब्लॉक को लागू करें। यह पहले परिभाषित एम्बेडिंग लेयर के आउटपुट को लेगा:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

अब, हम पूरा ट्रांसफॉर्मर मॉडल परिभाषित करने के लिए तैयार हैं:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT ट्रांसफॉर्मर मॉडल्स

**BERT** (Bidirectional Encoder Representations from Transformers) एक बहुत बड़ा मल्टी-लेयर ट्रांसफॉर्मर नेटवर्क है, जिसमें *BERT-base* के लिए 12 लेयर्स और *BERT-large* के लिए 24 लेयर्स होती हैं। इस मॉडल को पहले बड़े टेक्स्ट डेटा (WikiPedia + किताबें) के कॉर्पस पर अनसुपरवाइज्ड ट्रेनिंग (एक वाक्य में छुपे हुए शब्दों की भविष्यवाणी करना) का उपयोग करके प्री-ट्रेन किया जाता है। प्री-ट्रेनिंग के दौरान, मॉडल भाषा को समझने की एक महत्वपूर्ण क्षमता विकसित करता है, जिसे फिर अन्य डेटा सेट्स के साथ फाइन-ट्यूनिंग के जरिए उपयोग किया जा सकता है। इस प्रक्रिया को **ट्रांसफर लर्निंग** कहा जाता है। 

![http://jalammar.github.io/illustrated-bert/ से ली गई तस्वीर](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

BERT, DistilBERT, BigBird, OpenGPT3 और अन्य जैसे ट्रांसफॉर्मर आर्किटेक्चर के कई प्रकार हैं, जिन्हें फाइन-ट्यून किया जा सकता है। 

आइए देखें कि हम प्री-ट्रेन किए गए BERT मॉडल का उपयोग करके अपनी पारंपरिक सीक्वेंस क्लासिफिकेशन समस्या को कैसे हल कर सकते हैं। हम [आधिकारिक डाक्यूमेंटेशन](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) से विचार और कुछ कोड उधार लेंगे।

प्री-ट्रेन किए गए मॉडल्स को लोड करने के लिए, हम **Tensorflow hub** का उपयोग करेंगे। सबसे पहले, आइए BERT-विशिष्ट वेक्टराइज़र लोड करें:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

यह ज़रूरी है कि आप वही vectorizer इस्तेमाल करें जो मूल नेटवर्क पर ट्रेनिंग के दौरान उपयोग किया गया था। साथ ही, BERT vectorizer तीन घटक लौटाता है:
* `input_word_ids`, जो इनपुट वाक्य के लिए टोकन नंबरों का अनुक्रम है
* `input_mask`, जो दिखाता है कि अनुक्रम का कौन सा हिस्सा वास्तविक इनपुट है और कौन सा padding है। यह `Masking` लेयर द्वारा बनाए गए मास्क के समान है
* `input_type_ids` भाषा मॉडलिंग कार्यों के लिए उपयोग किया जाता है, और एक अनुक्रम में दो इनपुट वाक्यों को निर्दिष्ट करने की अनुमति देता है।

इसके बाद, हम BERT फीचर एक्सट्रैक्टर को instantiate कर सकते हैं:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


तो, BERT लेयर कई उपयोगी परिणाम लौटाती है:
* `pooled_output` पूरे अनुक्रम के सभी टोकन का औसत निकालने का परिणाम है। इसे पूरे नेटवर्क का एक बुद्धिमान अर्थपूर्ण एम्बेडिंग माना जा सकता है। यह हमारे पिछले मॉडल में `GlobalAveragePooling1D` लेयर के आउटपुट के समकक्ष है।
* `sequence_output` अंतिम ट्रांसफॉर्मर लेयर का आउटपुट है (जो हमारे ऊपर दिए गए मॉडल में `TransformerBlock` के आउटपुट के अनुरूप है)।
* `encoder_outputs` सभी ट्रांसफॉर्मर लेयर्स के आउटपुट हैं। चूंकि हमने 4-लेयर BERT मॉडल लोड किया है (जैसा कि आप शायद नाम से अनुमान लगा सकते हैं, जिसमें `4_H` शामिल है), इसमें 4 टेन्सर हैं। अंतिम टेन्सर `sequence_output` के समान है।

अब हम एंड-टू-एंड क्लासिफिकेशन मॉडल को परिभाषित करेंगे। हम *फंक्शनल मॉडल डिफिनिशन* का उपयोग करेंगे, जिसमें हम मॉडल का इनपुट परिभाषित करेंगे और फिर इसके आउटपुट की गणना के लिए एक श्रृंखला में अभिव्यक्तियाँ प्रदान करेंगे। हम BERT मॉडल के वेट्स को ट्रेन नहीं करेंगे और केवल अंतिम क्लासिफायर को ट्रेन करेंगे:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


हालांकि ट्रेन करने योग्य पैरामीटर बहुत कम हैं, प्रक्रिया काफी धीमी है क्योंकि BERT फीचर एक्सट्रैक्टर गणनात्मक रूप से भारी है। ऐसा लगता है कि हम उचित सटीकता प्राप्त करने में असमर्थ रहे, या तो प्रशिक्षण की कमी के कारण, या मॉडल पैरामीटर की कमी के कारण।

आइए BERT वेट्स को अनफ्रीज़ करें और इसे भी ट्रेन करें। इसके लिए बहुत छोटे लर्निंग रेट की आवश्यकता होती है, और साथ ही **वार्मअप** के साथ अधिक सावधानीपूर्वक प्रशिक्षण रणनीति की आवश्यकता होती है, जिसमें **AdamW** ऑप्टिमाइज़र का उपयोग किया जाता है। हम `tf-models-official` पैकेज का उपयोग करके ऑप्टिमाइज़र बनाएंगे:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

जैसा कि आप देख सकते हैं, प्रशिक्षण काफी धीमी गति से होता है - लेकिन आप कुछ epochs (5-10) के लिए मॉडल को प्रशिक्षित करने का प्रयास कर सकते हैं और देख सकते हैं कि क्या आप पहले उपयोग किए गए तरीकों की तुलना में सबसे अच्छा परिणाम प्राप्त कर सकते हैं।

## Huggingface Transformers लाइब्रेरी

Transformer मॉडल का उपयोग करने का एक और बहुत सामान्य (और थोड़ा सरल) तरीका [HuggingFace पैकेज](https://github.com/huggingface/) है, जो विभिन्न NLP कार्यों के लिए सरल बिल्डिंग ब्लॉक्स प्रदान करता है। यह Tensorflow और PyTorch, एक अन्य बहुत लोकप्रिय न्यूरल नेटवर्क फ्रेमवर्क, दोनों के लिए उपलब्ध है।

> **Note**: यदि आप यह देखने में रुचि नहीं रखते कि Transformers लाइब्रेरी कैसे काम करती है - तो आप इस नोटबुक के अंत तक जा सकते हैं, क्योंकि आप ऊपर किए गए कार्यों से कुछ भी मौलिक रूप से अलग नहीं देखेंगे। हम BERT मॉडल को प्रशिक्षित करने के उन्हीं चरणों को दोहराएंगे, लेकिन एक अलग लाइब्रेरी और काफी बड़े मॉडल का उपयोग करेंगे। इसलिए, प्रक्रिया में कुछ लंबा प्रशिक्षण शामिल है, तो आप केवल कोड को देख सकते हैं।

आइए देखें कि हमारा समस्या [Huggingface Transformers](http://huggingface.co) का उपयोग करके कैसे हल की जा सकती है।


सबसे पहले हमें उस मॉडल को चुनना होगा जिसे हम उपयोग करने जा रहे हैं। कुछ बिल्ट-इन मॉडल्स के अलावा, Huggingface में एक [ऑनलाइन मॉडल रिपॉजिटरी](https://huggingface.co/models) भी है, जहां आपको समुदाय द्वारा बनाए गए और भी कई प्री-ट्रेंड मॉडल मिल सकते हैं। इन सभी मॉडलों को केवल मॉडल का नाम देकर लोड और उपयोग किया जा सकता है। मॉडल के लिए आवश्यक सभी बाइनरी फाइल्स स्वचालित रूप से डाउनलोड हो जाएंगी।

कुछ स्थितियों में आपको अपने खुद के मॉडल लोड करने की आवश्यकता हो सकती है। ऐसे मामलों में, आप उस डायरेक्टरी को निर्दिष्ट कर सकते हैं जिसमें सभी संबंधित फाइल्स मौजूद हों, जैसे कि टोकनाइज़र के पैरामीटर, `config.json` फाइल जिसमें मॉडल पैरामीटर हों, बाइनरी वेट्स आदि।

मॉडल के नाम से, हम मॉडल और टोकनाइज़र दोनों को इंस्टैंशिएट कर सकते हैं। चलिए टोकनाइज़र से शुरू करते हैं:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` ऑब्जेक्ट में `encode` फ़ंक्शन होता है जिसे सीधे टेक्स्ट को एन्कोड करने के लिए उपयोग किया जा सकता है:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

हम टोकनाइज़र का उपयोग अनुक्रम को इस प्रकार एन्कोड करने के लिए भी कर सकते हैं जो मॉडल को पास करने के लिए उपयुक्त हो, जैसे `token_ids`, `input_mask` फ़ील्ड्स आदि शामिल करना। हम यह भी निर्दिष्ट कर सकते हैं कि हम Tensorflow टेन्सर चाहते हैं, इसके लिए `return_tensors='tf'` तर्क प्रदान कर सकते हैं:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

हमारे मामले में, हम एक पूर्व-प्रशिक्षित BERT मॉडल का उपयोग करेंगे जिसे `bert-base-uncased` कहा जाता है। *Uncased* का मतलब है कि यह मॉडल केस-सेंसिटिव नहीं है।

मॉडल को प्रशिक्षित करते समय, हमें टोकनाइज़ किए गए अनुक्रम को इनपुट के रूप में प्रदान करना होता है, और इसलिए हम डेटा प्रोसेसिंग पाइपलाइन डिज़ाइन करेंगे। चूंकि `tokenizer.encode` एक Python फ़ंक्शन है, हम इसे पिछले यूनिट की तरह ही उपयोग करेंगे, जिसमें इसे `py_function` का उपयोग करके कॉल किया जाएगा:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

अब हम `BertForSequenceClassification` पैकेज का उपयोग करके वास्तविक मॉडल लोड कर सकते हैं। यह सुनिश्चित करता है कि हमारे मॉडल में पहले से ही वर्गीकरण के लिए आवश्यक संरचना है, जिसमें अंतिम वर्गीकर्ता भी शामिल है। आपको एक चेतावनी संदेश दिखाई देगा जिसमें कहा जाएगा कि अंतिम वर्गीकर्ता के वज़न प्रारंभिक नहीं किए गए हैं, और मॉडल को पूर्व-प्रशिक्षण की आवश्यकता होगी - यह पूरी तरह से ठीक है, क्योंकि यही हम करने वाले हैं!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


जैसा कि आप `summary()` से देख सकते हैं, मॉडल में लगभग 110 मिलियन पैरामीटर हैं! संभवतः, यदि हम अपेक्षाकृत छोटे डेटासेट पर सरल वर्गीकरण कार्य करना चाहते हैं, तो हम BERT बेस लेयर को प्रशिक्षित नहीं करना चाहेंगे:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


अब हम प्रशिक्षण शुरू करने के लिए तैयार हैं!

> **नोट**: पूर्ण-स्तरीय BERT मॉडल का प्रशिक्षण करना बहुत समय लेने वाला हो सकता है! इसलिए हम इसे केवल पहले 32 बैचों के लिए प्रशिक्षित करेंगे। यह केवल यह दिखाने के लिए है कि मॉडल प्रशिक्षण कैसे सेट किया जाता है। यदि आप पूर्ण-स्तरीय प्रशिक्षण आज़माने में रुचि रखते हैं - तो बस `steps_per_epoch` और `validation_steps` पैरामीटर हटा दें, और इंतजार करने के लिए तैयार रहें!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


यदि आप iterations की संख्या बढ़ाते हैं, पर्याप्त समय तक प्रतीक्षा करते हैं, और कई epochs तक प्रशिक्षण करते हैं, तो आप उम्मीद कर सकते हैं कि BERT classification हमें सबसे अच्छी सटीकता प्रदान करेगा! इसका कारण यह है कि BERT पहले से ही भाषा की संरचना को काफी अच्छी तरह समझता है, और हमें केवल अंतिम classifier को fine-tune करने की आवश्यकता होती है। हालांकि, क्योंकि BERT एक बड़ा मॉडल है, पूरा प्रशिक्षण प्रक्रिया काफी समय लेती है और इसके लिए गंभीर computational शक्ति की आवश्यकता होती है! (GPU, और अधिमानतः एक से अधिक).

> **Note:** हमारे उदाहरण में, हमने सबसे छोटे pre-trained BERT मॉडल में से एक का उपयोग किया है। बड़े मॉडल उपलब्ध हैं जो संभवतः बेहतर परिणाम प्रदान कर सकते हैं।


## मुख्य बातें

इस यूनिट में, हमने **ट्रांसफॉर्मर्स** पर आधारित हाल ही की मॉडल आर्किटेक्चर देखी हैं। हमने इन्हें अपने टेक्स्ट वर्गीकरण कार्य के लिए लागू किया है, लेकिन इसी तरह, BERT मॉडल का उपयोग एंटिटी एक्सट्रैक्शन, प्रश्न उत्तर देने और अन्य NLP कार्यों के लिए भी किया जा सकता है।

ट्रांसफॉर्मर मॉडल NLP में वर्तमान में सबसे उन्नत तकनीक का प्रतिनिधित्व करते हैं, और अधिकांश मामलों में, यह वह पहला समाधान होना चाहिए जिसके साथ आप कस्टम NLP समाधान लागू करते समय प्रयोग करना शुरू करें। हालांकि, यदि आप उन्नत न्यूरल मॉडल बनाना चाहते हैं, तो इस मॉड्यूल में चर्चा किए गए पुनरावर्ती न्यूरल नेटवर्क के मूलभूत सिद्धांतों को समझना अत्यंत महत्वपूर्ण है।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
